# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv file and read as dataframe
City_Data_file_path = "../WeatherPy/Weather_Output/City_Data.csv"
City_Data = pd.read_csv(City_Data_file_path)

# review new dataframe
City_Data.describe()

,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Date
count,602.000000,602.000000,602.000000,602.000000,602.000000,602.000000,6.020000e+02
mean,20.206196,15.822143,59.772807,72.805648,55.254153,8.366595,1.603662e+09
std,32.189304,90.587705,21.338792,20.559894,38.784702,5.870497,1.017221e+02
min,-54.800000,-175.200000,-2.200000,12.000000,0.000000,0.200000,1.603661e+09
25%,-6.220000,-65.897500,45.000000,64.000000,13.500000,4.075000,1.603662e+09
50%,20.990000,21.185000,66.180000,78.000000,69.000000,6.930000,1.603662e+09
75%,46.825000,94.475000,77.852500,88.000000,90.000000,11.410000,1.603662e+09
max,78.220000,179.320000,95.000000,100.000000,100.000000,46.350000,1.603662e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
'width': '1000px',
'height': '400px',
'border': '1px solid black',
'padding': '1px'
}

center_coordinates = City_Data[['Lat','Lng']].mean()

# Set variants for Lat and Lng as locations and Humidity as the weight
locations = City_Data[['Lat','Lng']]
humidity_weight = City_Data['Humidity']

fig = gmaps.figure(map_type='TERRAIN', center=center_coordinates, zoom_level=2, layout=figure_layout)

# Add Heatmap layer to map
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity_weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Drop all rows with null values
city_data_df = City_Data.dropna(how="any")
city_data_df.describe()

,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Date
count,598.000000,598.000000,598.000000,598.000000,598.000000,598.000000,5.980000e+02
mean,20.509047,15.827007,59.771271,72.759197,55.456522,8.343796,1.603662e+09
std,32.081465,90.890630,21.408652,20.584398,38.756171,5.837366,1.019224e+02
min,-54.800000,-175.200000,-2.200000,12.000000,0.000000,0.200000,1.603661e+09
25%,-6.120000,-66.275000,45.000000,64.000000,15.000000,4.097500,1.603662e+09
50%,21.375000,22.110000,66.200000,78.000000,69.500000,6.930000,1.603662e+09
75%,47.470000,94.767500,77.880000,88.000000,90.000000,11.410000,1.603662e+09
max,78.220000,179.320000,95.000000,100.000000,100.000000,46.350000,1.603662e+09


In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#     A max temperature lower than 80 degrees but higher than 75
#     Wind speed less than 10 mph
#     Zero cloudiness

ideal_city = city_data_df.loc[(city_data_df['Max Temp °F']<80) & (city_data_df['Max Temp °F']>75) &
                               (city_data_df['Wind Speed']<10) & (city_data_df['Cloudiness']== 0)]
# Display dataframe
ideal_city

,City,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Country,Date
136,Kiryat Gat,31.61,34.76,75.20,83,0,4.70,IL,1603662246
178,Cabedelo,-6.98,-34.83,78.80,83,0,6.93,BR,1603662198
218,Maceió,-9.67,-35.74,77.00,83,0,6.93,BR,1603662277
300,Pitimbu,-7.47,-34.81,78.80,83,0,6.93,BR,1603662311
318,Morondava,-20.28,44.28,75.97,61,0,9.78,MG,1603662109
387,Beira,-19.84,34.84,75.20,88,0,9.17,MZ,1603662348
413,Salalah,17.02,54.09,77.00,57,0,5.82,OM,1603662361
534,Makkah al Mukarramah,21.43,39.83,77.88,69,0,2.86,SA,1603662220
541,Hobyo,5.35,48.53,79.41,83,0,8.59,SO,1603662413
549,Tessalit,20.20,1.01,79.59,15,0,9.51,ML,1603662416


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel_df to be updated with Hotel Names
hotel_df = ideal_city

# Setup For Loop
for index, row in hotel_df.iterrows():
    coordinates = f"{row['Lat']}, {row['Lng']}"

# Get params and set Error Handler
    try:
        for hotel in ideal_city:
            params = {
                "location": coordinates,
                "type": "lodging",
                "radius": 5000,
                "key": g_key
            }
# Base URL
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using params dictionary
        response = requests.get(base_url, params=params).json()

# extract results
        results = response['results']
        ihotel_name_val = results[0]["name"]

# Assign Hotel Name (if available) to hotel_df
        hotel_df.at[index,'Hotel Name'] = ihotel_name_val
    except:
        hotel_df.at[index,'Hotel Name'] = "Hotel Not Found"

C:\Users\anaro\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\anaro\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
# Display updated dataframe with 'Hotel Name'
hotel_df

,City,Lat,Lng,Max Temp °F,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
136,Kiryat Gat,31.61,34.76,75.20,83,0,4.70,IL,1603662246,OlusHome
178,Cabedelo,-6.98,-34.83,78.80,83,0,6.93,BR,1603662198,Lua Mansa Pousada
218,Maceió,-9.67,-35.74,77.00,83,0,6.93,BR,1603662277,Mercure Maceio Pajucara Hotel
300,Pitimbu,-7.47,-34.81,78.80,83,0,6.93,BR,1603662311,Reserva do Abiaí
318,Morondava,-20.28,44.28,75.97,61,0,9.78,MG,1603662109,La Case Bambou
387,Beira,-19.84,34.84,75.20,88,0,9.17,MZ,1603662348,Beira Terrace Hotel
413,Salalah,17.02,54.09,77.00,57,0,5.82,OM,1603662361,HAMDAN PLAZA HOTEL
534,Makkah al Mukarramah,21.43,39.83,77.88,69,0,2.86,SA,1603662220,"Makkah Clock Royal Tower, A Fairmont Hotel"
541,Hobyo,5.35,48.53,79.41,83,0,8.59,SO,1603662413,Halane Hotel and Restaurent
549,Tessalit,20.20,1.01,79.59,15,0,9.51,ML,1603662416,Hotel Not Found


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Create empty list to hold varibles for info_box_template
ideal_hotel_info = []

# Pull varibales from hotel_df and populate ideal_hotel_info list
for index, rows in hotel_df.iterrows():
    ideal_hotel_info.append(
        {
            "Hotel Name": rows['Hotel Name'],
            "City": rows['City'],
            "Country": rows['Country'],
            "Location": (rows['Lat'], rows['Lng'])
        }
    )

# Create list with hotel locations
hotel_location = [hotels['Location'] for hotels in ideal_hotel_info]

In [10]:
# Add marker layer ontop of heat map
hotel_info = [info_box_template.format(**hotel) for hotel in ideal_hotel_info]

markers = gmaps.marker_layer(hotel_location, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='1000px'))